# 文本分类挖掘

机器学习中，分类问题是基础，很多应用都可以转化为分类问题，本实验重点试验文本分类。而针对中文文本数据，往往需要进行预处理和中文分词，中文分词不准确，将造成偏差过高。因此在对社交媒体文本数据训练模型之前需要数据预处理与中文分词。

本章实验分别进行文本数据预处理、中文分词和分类。

本章实验涉及KNN、SVM等分类算法，通过不同算法的效果对比来加深对算法的理解。

## 实验目的

    	掌握文本数据预处理的常用方法
    	掌握中文分词的常用工具
    	掌握KNN、SVM、决策树、高斯贝叶斯模型、伯努利贝叶斯模型等分类算法的原理与使用
    	掌握模型评估的方法


## 先读取相关的EXCEL文件

**pandas.read_excel(io,sheetname=0,header=0,names=None,index_col=None,usecols=None,squeeze= False,dtype=None,engine=None,converters=None,true_values=None,false_values=None,skiprows= None,nrow=None,na_values=None,keep_default_na=True,verbose=False,parse_dates=False, date_parser=None,thousands=None,comment=None,skipfooter=0,conver_float=True, mangle_dupe_cols=True)**

    io ：字符串，xls 或 xlsx 文件路径或类文件对象。
    sheet_name ：None、字符串、整数、字符串列表或整数列表，默认值为 0。字符串用于工作表名称；整数为索引，表示工作表位置，字符串列表或整数列表用于请求多个工作表，为 None 时则获取所有的工作表。参数值如表4 所示。
    header ：指定作为列名的行，默认值为 0，即取第一行的值为列名。数据为除列名以外的数据；若数据不包含列名，则设置为 header=None。
    names ：默认值为 None，要使用的列名列表。
    index_col ：指定列为索引列，默认值为 None，索引 0 是 DataFrame 对象的行标签。
    usecols ：int、list 或字符串，默认值为 None。
    如果为 None，则解析所有列。
    如果为 int，则解析最后一列。
    如果为 list 列表，则解析列号和列表的列。
    如果为字符串，则表示以逗号分隔的 Excel 列字母和列范围列表（例如，“A ：E”或    “A，C，E ：F”），范围包括双方。
    squeeze ：布尔值，默认值为 False，如果解析的数据只包含一列，则返回一个 Series。
    dtype ：列的数据类型名称或字典，默认值为 None。例如，{ 为 'a' ：np.float64，'b' ：np.int32}。
    kiprows ：省略指定行数的数据，从第一行开始。
    skipfooter ：省略指定行数的数据，从尾部数的行开始。

![](Pic/sheet.JPG)

说明：如果设置sheet_name = None, 则在生成表格的时候是返回的是dict字典类型，

因此如果想要重新处理成pandas.DataFrame类型需要修正方法，使用pandas.concat()方法对数据进行批量的连接

**pd.concat(objs, axis=0, join='outer', join_axes=None, ignore_index=False,
    keys=None, levels=None, names=None, verify_integrity=False)**

    objs: series，dataframe或者是panel构成的序列lsit
    axis： 需要合并链接的轴，0是行，1是列
    join：连接的方式 inner，或者outer
![](Pic/2t.png)

In [ ]:
import pandas as pd
import collections
df_dict0 = pd.read_excel('新冠00.xlsx', sheet_name=None, index_col=None, na_values=['NA'])
df_dict0 = collections.OrderedDict(sorted(df_dict0.items()))
df = pd.concat(df_dict0.values(), ignore_index=True)


df_dict = pd.read_excel('新冠01.xlsx', sheet_name=None, index_col=None, na_values=['NA'])
df_dict = collections.OrderedDict(sorted(df_dict.items()))
dfx = pd.concat(df_dict.values(), ignore_index=True)

dfx = pd.concat([dfx, df], ignore_index=True)
dfx

In [ ]:
df_ym = pd.read_excel('new疫苗.xlsx', sheet_name=None, index_col=None, na_values=['NA'])
df_ym = collections.OrderedDict(sorted(df_ym.items()))
df_data = pd.concat(df_ym.values(), ignore_index=True)
df_data

本次实验A，个人使用了两个文本，拟分类为两部分：“新冠”模型和“疫苗“模型。 添加LB列表示标记，LB=0 表示新冠， LB=1表示疫苗。进行挖掘

实验B是小组合作的实验，实验根据之前爬取微博的结果，总体大致定义有4类。由于爬取的搜索关键词不同，得到的结果也不尽相同，现在打算对它们进行分类

第一类： 明星偶像类，以“肖战”和“王一博”为主要关键词作获取得到的结果，总共有

第二类： 新闻事件类，以“新冠”和“疫苗”为主要关键词获取得到的结果，结合当下的情境挖掘，总共有10376条的数据

第三类： 游戏类，以“原神”为关键词挖掘相关信息，总共有

第四类： 体育明星类，以“马龙”为关键词，总共有

### TEST FOR LAB A

下面我们尝试对相关内容进行标记添加，这里样例是针对LAB A,两个xlsx文件添加一列数据，
分类0或1并合并数据

添加

In [ ]:
dfx['LabelA'] = 0
dfx.head()

同理，对第二个表格标记B，并合并两个表格

In [ ]:
df_data['LabelA'] = 1
whole_xlsx = dfx = pd.concat([dfx, df_data], ignore_index=True)
whole_xlsx

可以看到对应的总数据量有1000多条，对着1w多条数据进行分类，分类之前需要预处理

### 文本数据预处理

首先我们进行数据清洗，对无意义的数据处理和数据去除

In [ ]:
import numpy as np
import pandas as pd

# dropping columns that aren't needed
# 删除指定的学生成绩数据，关键代码如下：
# df.drop(['数学'],axis=1,inplace=True) #删除某列
#
# df.drop(columns='数学',inplace=True) #删除columns为“数学”的列
#
# df.drop(labels='数学', axis=1,inplace=True) #删除列标签为“数学”的列
whole_xlsx.drop(whole_xlsx.columns[[0,2,3,5,8,9,10,11]], axis=1, inplace=True)
whole_xlsx

可以看到相关无关的列被删除了，剩下了博主主页、微博内容、发布时间、微博地址和标签列

下面我们需要把相关缺失信息的行去掉。实验中是对微博内容为空的行去掉

**DataFrame.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)**

关键参数详解：

    axis=0/1，默认为0。axis=0代表d对行数据进行操作，axis=1代表列数据。
    how=any/all，默认为any。how=any代表若某行或某列中存在缺失值，则删除该行或该列。
    how=all:若某行或某列中数值全部为空，则删除该行或该列。
    thresh=N，可选参数，代表若某行或某列中至少含有N个缺失值，则删除该行或该列。
    subset=列名，可选参数，代表若指定列中有缺失值，则删除该行。
    inplace=True/False，Boolean数据, 默认为False。inplace=True代表直接对原数据集N做出修改。
    inplace=False代表修改后生成新数据集M，原数据集N保持不变。

注意：处理行之后的对应的数据行索引会出错，这里我们还要进行索引重置，即用到reset_index方法实现

In [ ]:
del_whole_xlsx=whole_xlsx.dropna(axis=0, subset= ["微博内容","微博地址"])
del_whole_xlsx.reset_index(drop=True,inplace=True)#drop=True：删除原行索引；inplace=True:在数据上进行更新
del_whole_xlsx.isnull().any()

可以看到每个维度的数据不再是空值了。

下面进行去除或替换一些无效或不需要的信息，比如特殊字符、标点符号等。

In [ ]:
# dropping invalid character
del_whole_xlsx['微博内容'] = del_whole_xlsx['微博内容'].str.replace(r'[^\w]+', '')
del_whole_xlsx.head()

可以明显地看到微博内容的数据明显变得结构化、标准化了，便于后期进行机器学习

下面我们对这些数据进行重复值处理，再观察留下的数据有多少

#### 重复值处理

重复值处理的方法有：

    1.字符串匹配去重：drop_duplicates()；
    2.基于中文分词后的关键词字符串去重：先将文本数据转化为关键词字符串，再使用字符串匹配去重。（此方法需要使用中文分词工具，将在3.3.2中介绍）
    3.不处理

对于本次实验的数据集和任务，应该对于重复值如何处理呢？

pandas中，除去重复值的常用方法为drop_duplicate，其基本格式如下：

**DataFrame.drop_duplicates(subset=None, keep='first', inplace=False)：**

关键参数详解：

    subset:用来指定特定的列，默认所有列；
    keep: {'first', 'last', False}。默认值为'first'，用于删除重复项并保留第一次出现的项；
    inplace:是直接在原来数据上修改还是保留一个副本，默认是False

同理，重复值去除后的数据需要进一步处理

In [ ]:
del_rept = del_whole_xlsx.drop_duplicates(['微博内容', '微博地址'],keep='last')
del_rept.reset_index(drop=True,inplace=True)#drop=True：删除原行索引；inplace=True:在数据上进行更新
del_rept

## 中文分词研究

### 步骤 1	中文分词
分词就是将一段连续的文本按照一定的规范重新组合成词序列的过程。

我们知道，在英文的行文中，单词之间是以空格作为自然分界符的，而中文只是字、
句和段能通过明显的分界符来简单划界，唯独词没有一个形式上的分界符，
虽然英文也同样存在短语的划分问题，不过在词这一层上，中文比之英文要复杂得多、困难得多。

常见的中文分词工具有许多，本实验使用jieba库用于中文分词。

In [ ]:
import jieba
data = del_rept['微博内容'].values
lb = del_rept['LabelA'].values
segdata = []
for line in data:
    seglist=jieba.cut(line, cut_all=False)
    segdata.append('|'.join(seglist))
segdata

可以看到得到分词处理后的相关文本。

### 步骤 2	去停用词

在进行基于关键词向量去重或者文本分类任务之前，
需要先去掉与任务无关的停用词、高频词等。

停用词是指在信息检索中，为节省存储空间和提高搜索效率，在处理自然语言数据（或文本）之前或之后会自动过滤掉某些字或词，这些字或词即被称为Stop Words（停用词）。
这些停用词都是人工输入、非自动化生成的，生成后的停用词会形成一个停用词表。

去停用词对应的gitHub网址如下：[https://github.com/goto456/stopwords](https://github.com/goto456/stopwords)

下载相关所需的stepwords.txt文件执行文本相关的匹配

例如，试验的去停用词代码如下：

In [ ]:
words = jieba.cut("我是厦门大学软件工程的学生",cut_all=False)
stopwords = []
seglist = ""
for word in open('stopwords/baidu_stopwords.txt','r',encoding='utf-8'):
    stopwords.append(word.strip())
for word in words:
    if word not in stopwords:
        seglist += word + "|"
print(seglist)


注意上述打开相关文件要进行编码转换，换成utf-8，否则会报错UnicodeError

可以看到明显的停用词被去掉了，保留了比较关键的信息。

jieba.cut 方法接受三个输入参数:

    sentence 需要分词的字符串；
    cut_all  参数用来控制是否采用全模式；cut_all = Ture 表示全模式，False表示精准模式
    HMM 参数用来控制是否使用HMM模型

### 步骤 3	保存预处理后的文件

In [ ]:
#字典中的key值即为csv中列名
dataframe = pd.DataFrame({'微博内容':data,'类标签':lb})
#将DataFrame存储为csv,index表示是否显示行名，default=True
dataframe.to_csv("weibo-processed-A.csv",index=False,sep=',')
print('Successfully saved!')


本实验我们对微博数据进行了数据预处理，
主要包括数据清洗、重复值处理与中文分词。

到此预处理结束，下面进行文本分类挖掘

### LAB A 文本挖掘

本次实验我们将使用KNN算法对社交媒体文本数据集进行相应的分类。

KNN算法是一种分类算法，1968年由Cover和Hart提出，其基本思想为：一个样本与数据集中的k个样本最相似，

如果这k个样本中的大多数属于某一个类别，则该样本也属于这个类别。

#### 实验内容

本次实验使用的基于第3节预处理和中文分词后的文本数据集weibo-processed-A.csv格式文件。

### 文本分类 —— 数据读取

In [ ]:
#数据读取
df_post = pd.read_csv(r"weibo-processed-A.csv",encoding="utf8")
df_post

### 数据集划分

执行下面方框中的这段代码，
可以导入本次实验中使用的Python开发基本工具库。

sklearn是一个机器学习库，这里导入了train_test_split用于数据集划分，
导入了cross_val_score用于模型评估。

CountVectorizer是属于常见的特征数值计算类，是一个文本特征提取方法。对于每一个训练文本，它只考虑每种词汇在该训练文本中出现的频率。

CountVectorizer会将文本中的词语转换为词频矩阵，它通过fit_transform函数计算各个词语出现的次数。

In [ ]:
import itertools
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix  # 系数矩阵建立
# from sklearn.decomposition import PCA        # PCA 降维
# from sklearn.model_selection import cross_val_score,train_test_split  # 数据集验证和划分

#### 设置停用词，构造词频矩阵

In [ ]:
#设置停用词，构建词频矩阵
stopwords = []
for word in open('stopwords/cn_stopwords.txt','r',encoding='utf-8'):
    stopwords.append(word.strip())
def tokenizer(s):
    words=[]
    cut = jieba.cut(s)
    for word in cut:
        words.append(word)
    return words
count = CountVectorizer(tokenizer=tokenizer, stop_words=list(stopwords))
countvector = count.fit_transform(df_post['微博内容']).toarray()
c = 0
print(countvector)
print(countvector.shape)  #打印看看生成的词频矩阵相关参数
for i in countvector[0]:
    if i==1:
        c+=1
c

可以看到矩阵两个维度都比较大，而且矩阵并不是全是0的矩阵

下面我们尝试使用PCA降维将词频矩阵转化为二维数据并作图

In [ ]:
# 使用PCA降维将词频矩阵转化为二维数据，画图
from sklearn.decomposition import PCA        # PCA 降维

kind = np.unique(df_post['类标签'].values)
print(kind)
pca = PCA(n_components=2)
newvector = pca.fit_transform(countvector)
plt.figure()
for i,c,m in zip(range(len(kind)),['r','b','g','y'],['o','^','>','<']):
    index = df_post[df_post['类标签']==kind[i]].index
    x = newvector[index,0]
    y = newvector[index,1]
    plt.scatter(x,y,c=c,marker=m,label=kind[i])
plt.legend()
plt.xlabel('X Label')
plt.ylabel('Y Label')
plt.show()
plt.savefig('PCAA.png',dpi=600)

注意：上面的代码运行需要比较长的时间，对更大的数据考虑使用华为云平台运行

### 步骤 3	数据集划分
数据集划分的方法有：

    1.留出法；
    2.交叉验证法；
    3.自助法。
对于本次实验的数据集和任务，应该用什么方法处理呢？
    本次实验以留出法为例：
    打印数据集标签统计数据，代码：

In [ ]:
y = df_post['类标签']
print(y)
#X为去掉标签列的数据
X = countvector
print(X)
print("X.shape", X.shape)
print("y.shape", y.shape)

测试，参考个人的gitHub链接：[https://github.com/JunTheRipper/Data-Mining-Access/blob/main/3.Data-Classification-Forecast/3.Data-Classification-Forecast%E6%95%B0%E6%8D%AE%E5%88%86%E7%B1%BB%E9%A2%84%E6%B5%8B.ipynb](https://github.com/JunTheRipper/Data-Mining-Access/blob/main/3.Data-Classification-Forecast/3.Data-Classification-Forecast%E6%95%B0%E6%8D%AE%E5%88%86%E7%B1%BB%E9%A2%84%E6%B5%8B.ipynb)

利用train_test_split方法，将X,y随机划分为训练集（train_data），训练集标签（train_labels），测试集（test_data），试集标签（test_labels），按训练集：测试集=8:2的概率划分。


In [ ]:
from sklearn.model_selection import train_test_split
train_data, test_data, train_labels, test_labels = train_test_split(X, y, test_size=0.2,random_state=1)
print("train_data.shape", train_data.shape)
print("test_data.shape", test_data.shape)
print("train_labels.shape", train_labels.shape)
print("test_labels.shape", test_labels.shape)

### 步骤 7	KNN分类

参考链接：[https://github.com/JunTheRipper/Data-Mining-Access/blob/main/Summary-Of-Machine-Learning/B.sipervised-learning/B1.classification.ipynb](https://github.com/JunTheRipper/Data-Mining-Access/blob/main/Summary-Of-Machine-Learning/B.sipervised-learning/B1.classification.ipynb)


In [ ]:
from sklearn.neighbors import KNeighborsClassifier

kNN_classifier = KNeighborsClassifier(n_neighbors=5)#初始化k近邻算法对象
kNN_classifier.fit(train_data,train_labels)#对训练集进行训练
label_predict = kNN_classifier.predict(test_data)#对测试集进行预测
"""预测结果"""
print("Predict_rlt:",label_predict)
"""预测准确个数"""
print("Correct_no:",sum(label_predict==test_labels))
"""预测准确率"""
print("Accuracy:",sum(label_predict==test_labels)/len(test_labels))


### 模型评估

这里用交叉验证法进行模型评估：

In [ ]:
from sklearn.model_selection import cross_val_score
knn = KNeighborsClassifier(n_neighbors=5)
score_knn_accuracy = np.mean(cross_val_score(estimator=knn, X=X, y=y, cv=5, scoring='accuracy'))
print("Score_accuracy:",score_knn_accuracy)

可以看到上面的输出

## TEST FOR LAB B

下面的项目是团队的项目，同样我们用上面的方法进行处理。
注意，数据量比较大，选择的情况都有影响！

首先我们封装成两个函数，进行多个表格进行合并处理

In [ ]:
import pandas as pd
import collections
def concat_xlsx(filename):
    '''
    :param filename:
    :return: df
    :function: 实现一个表格的多sheet的合并
    '''
    df_dict0 = pd.read_excel(filename, sheet_name=None, index_col=None, na_values=['NA'])
    df_dict0 = collections.OrderedDict(sorted(df_dict0.items()))
    df = pd.concat(df_dict0.values(), ignore_index=True)
    return df

def tables_xlsx_concat(df1,df2):
    '''
    :param df1:
    :param df2:
    :return: dfx
    :function 实现两个表格数据合并（前提是两个表格维度相同）
    '''
    dfx = pd.concat([df1, df2], ignore_index=True)
    return dfx


def generate_data_needed(datafm):
    '''
    function:实现对所需要的维度的提取，在这里我们只保留微博内容和微博地址两项
    :param datafm: pd.DataFrame
    :return: dealt_df
    '''
    return datafm[['微博内容','微博地址']]


def add_labels(df,label):
    '''
    function:为相关的DataFrame格式的项目打上标签
    :param df:
    :param label:
    :return df:
    '''
    df['标签']  =  label
    return df

if __name__ == '__main__':
    label_file_MaLong = 'Data/0/data.xlsx'
    label_file_Stars_1 = 'Data/1/WYB.xlsx'
    label_file_Stars_2 = 'Data/1/WYB2.xlsx'
    # label_file_Stars_3 = 'Data/1/XZ.xlsx'
    # label_file4 = ''
    # label_file5 = ''
    label_file_Game = 'Data/2/new原神.xlsx'
    label_file_COVID_1 = 'Data/3/new疫苗.xlsx'
    label_file_COVID_2 = 'Data/3/新冠00.xlsx'
    label_file_COVID_3 = 'Data/3/新冠01.xlsx'

# 4类的dataFrame
dataA = add_labels(generate_data_needed(concat_xlsx(label_file_MaLong)),0)

dataB = add_labels(generate_data_needed(
        tables_xlsx_concat(
            concat_xlsx(label_file_Stars_1),concat_xlsx(label_file_Stars_2))),1)

dataC = add_labels(generate_data_needed(concat_xlsx(label_file_Game)),2)

dataD = add_labels(generate_data_needed(
    tables_xlsx_concat(
        tables_xlsx_concat(
            concat_xlsx(label_file_COVID_1),
            concat_xlsx(label_file_COVID_2)),
        concat_xlsx(label_file_COVID_3))),3)


whole_data = pd.concat([dataA, dataB, dataC, dataD], ignore_index=True)
print(whole_data.shape)
whole_data

简单说明一下对上面的多个xlsx文本处理模式

    1 先读取xlsx的文件数据，再转换为pandas内置pd.DataFrame数据类型
    2 对多sheet的文本进行sheet读取合并技术
    3 对多个xlsx转换后的DataFrame类型进行处理，实现多表并成1表
    4 数据预清洗，保留需要的关键信息数据
    5 打上标签
    6 最后多类别数据合并成单个DataFrame

下一步进行数据清洗，对无意义的数据处理和数据去除

先观察一下是否有缺失值，如果有，把相关空的行去掉，注意索引的修正

In [ ]:
whole_data.isnull().any()

In [ ]:
whole_data=whole_data.dropna(axis=0, subset= ["微博内容"])
# whole_data=whole_data.dropna(axis=0, subset= ["微博地址"])
whole_data.reset_index(drop=True,inplace=True)#drop=True：删除原行索引；inplace=True:在数据上进行更新
whole_data.isnull().any()

去除空行后，看到明显的空值数据被处理完毕

接下来，去除重复的数据，如果微博内容和地址重复，可以考虑去掉.同样的，我们可以考虑使用

In [ ]:
whole_data = whole_data.drop_duplicates(['微博内容', '微博地址'],keep='last')
whole_data.reset_index(drop=True,inplace=True)#drop=True：删除原行索引；inplace=True:在数据上进行更新
whole_data

可以看到数据得到了清洗。下面进行去除或替换一些无效或不需要的信息，比如特殊字符、标点符号等。


In [ ]:
whole_data['微博内容'] = whole_data['微博内容'].str.replace(r'[^\w]+', '')
whole_data.head()

### 中文分词研究

测试1 ：使用jieba库用于中文分词

In [ ]:
import time
import jieba
start =  time.perf_counter()

content_data = whole_data['微博内容'].values
label_data = whole_data['标签'].values
segdata = []
for line in content_data:
    seglist=jieba.cut(line, cut_all=False)
    segdata.append('|'.join(seglist))


end =time.perf_counter()
print("程序运行时间为：",end-start,"s")
print(segdata[0])

除了jieba分词，每个小组成员分别实现一种分词工具，请对比分析不同分词工具的结果。

#### 测试A：yaha库

通过简单定制，让分词模块更适用于你的需求。使用多线程，以及类似MapReduce的思想，可以处理50M+的文本，自动得到文本当中的专业名词、名字、地点名词等等词语。
得到词语后可以加到分词工库的字典中。

基本功能：

* 精确模式，将句子切成最合理的词。

* 全模式，所有的可能词都被切成词，不消除歧义。 * 搜索引擎模式，在精确的基础上再次驿长词进行切分，提高召回率，适合搜索引擎创建索引。

* 备选路径，可生成最好的多条切词路径，可在此基础上根据其它信息得到更精确的分词模式。

对于本实验的数据集和任务，应该对于重复值如何处理呢？

## 步骤 2	去停用词
在进行基于关键词向量去重或者文本分类任务之前，需要先去掉与任务无关的停用词、高频词等。例如，去停用词代码如下：

### 步骤 3	保存预处理后的文件

In [ ]:
#字典中的key值即为csv中列名
dataframe = pd.DataFrame({'微博内容':content_data,'类标签':label_data})

#将DataFrame存储为csv,index表示是否显示行名，default=True
dataframe.to_csv("weibo-processed-B.csv",index=False,sep=',')

print('Successfully saved!')


### 说明：

由于数据量达到5w,做接下来的文本分类的时候会导致出现内存超限或者时间复杂度过高的情况。

在这里我们尝试把5w条数据标签顺序随机打乱，分成10份，对每一份数据进行相关的处理观察结果

打乱顺序和分割的代码

In [ ]:
def shuffle_data(data):
    '''

    :param data: pandas.DataFrame
    :return shf_data: pandas.DataFrame
    '''
    shf_data = None
    return shf_data


def spilt_csv(shf_data):
    '''
    :param shf_data:
    :return None:
    '''
    pass